<center>
    <h1> Machine Learning Project: Weather Stations </h1> 
    <h2>By: Mohammad Traboulsi </h2>
    
    <h2>Presented to: Dr. Bruno deremble  </h2>
</center>

In [13]:
# import and default settings 
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from pathlib import Path
import folium as fl

%matplotlib inline
sns.set_context('talk')
sns.set_style('whitegrid')
mpl.rcParams['figure.figsize'] = (12, 8)

def add_format(**kwargs):
    'add default format to any figure'
    ax = plt.gca()
    for kw, item in kwargs.items():
        if kw == 'xlabel':
            ax.set_xlabel(item)
        elif kw == 'ylabel':
            ax.set_ylabel(item)
        elif kw == 'title':
            ax.set_title(item)
        elif kw == 'xlim':
            ax.set_xlim(item)
        elif kw == 'ylim':
            ax.set_ylim(item)
    ax.minorticks_on()
    ax.xaxis.grid(True, "minor", linewidth=.8, ls=':')
    ax.yaxis.grid(True, "minor", linewidth=.8, ls=':')
    plt.show()

DATA_PATH = Path('./data')

NAMES = {
    1: 'Paris',
    2: 'Bordeaux',
    3: 'Metz',
    4: 'Grenoble',
    5: 'Bretagne'
}
def read_file(file):  
    # read file and average over all points (13 points per file)
    columns = ['date', 'id', 'lat', 'lon', 'ts', 'tp', 'sh','ws' ]
    df = pd.read_csv(file, comment='#', sep=';', index_col=0, parse_dates=True, names=columns, na_values=-999.99)
    df = df.groupby(df.index).mean().loc[:'2030-12-31']
    # change ids to more useful ones 
    ids = dict(((6190,2),(14512,1), (15405,3), (7252, 4), (13461, 5)))
    idx = df['id'][1]
    df['id'] = ids[idx]
    return df

In [14]:
# read all files
df = pd.DataFrame()
for file in DATA_PATH.glob('*.txt'):
    temp= read_file(file)
    df = pd.concat([df, temp])
df

,id,lat,lon,ts,tp,sh,ws
date,,,,,,,
2006-01-01,2,44.617446,-0.332663,5.540000,0.000000,0.005092,1.064692
2006-01-02,2,44.617446,-0.332663,4.130769,0.169615,0.004794,1.260077
2006-01-03,2,44.617446,-0.332663,7.097692,0.520308,0.006998,1.725615
2006-01-04,2,44.617446,-0.332663,8.660000,0.000000,0.006824,1.137692
2006-01-05,2,44.617446,-0.332663,9.130000,0.000000,0.006722,0.685231
...,...,...,...,...,...,...,...
2030-12-27,1,48.823308,2.500530,11.002308,0.006154,0.007414,3.883538
2030-12-28,1,48.823308,2.500530,10.652308,5.701231,0.006998,5.632308
2030-12-29,1,48.823308,2.500530,12.589231,2.874615,0.008398,7.384846


In [18]:
coords = df.drop_duplicates(subset='id')[['id', 'lat', 'lon']].values
france = fl.Map(location=[47.122, 4.732], zoom_start=6)
for idx, lat,lon in coords:
    color, icon = 'green', 'cloud'
    if idx == 1:
        color, icon = 'red', 'info-sign'
    fl.Marker(
    location=[lat, lon],
    tooltip= NAMES[idx],
    icon=fl.Icon(color=color, icon=icon),
    ).add_to(france)
france